## Requisitos previos:
Antes de comenzar, asegúrese de que las siguientes bibliotecas estén instaladas en su entorno Python: chembl_webresource_client, pandas, numpy, rdkit, seaborn, matplotlib, sklearn, y bayes_opt.


### Paso 1: Importar bibliotecas y módulos
Abra su entorno de Python e importe todas las bibliotecas y módulos necesarios utilizando el código proporcionado.


In [ ]:
from chembl_webresource_client.new_client import new_client
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import Descriptors
from rdkit.Chem import MACCSkeys
from rdkit.Chem.AllChem import GetMorganFingerprintAsBitVect
from rdkit.Chem.AllChem import GetHashedTopologicalTorsionFingerprintAsBitVect
from rdkit.Chem import RDKFingerprint
from rdkit import DataStructs
import pickle
import numpy
import seaborn as sns
import matplotlib
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.model_selection import KFold
from sklearn.metrics import auc
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import roc_curve
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt

matplotlib.use('Agg')

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization
from sklearn.model_selection import train_test_split

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.metrics import confusion_matrix,classification_report,precision_score, recall_score, f1_score, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from bayes_opt import BayesianOptimization

from sklearn import metrics

from sklearn import metrics
import matplotlib.pyplot as plt
import numpy
from sklearn import metrics
import pickle
import os

### Paso 2: Definir funciones para cálculo de Fingerprints
En este paso, se definen las funciones calculate_fp y ConvertToNumpyArray. Estas funciones se utilizan para calcular y convertir los Fingerprints de las moléculas.


In [ ]:
def calculate_fp(mol, method='rdk5', n_bits=500):
    if method == 'maccs':
        return MACCSkeys.GenMACCSKeys(mol)
    if method == 'ecfp4':
        return GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits, useFeatures=False)
    if method == 'ecfp6':
        return GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits, useFeatures=False)
    if method == 'torsion':
        return GetHashedTopologicalTorsionFingerprintAsBitVect(mol, nBits=n_bits)
    if method == 'rdk5':
        return RDKFingerprint(mol, maxPath=5, fpSize=1024, nBitsPerHash=2)

def ConvertToNumpyArray(fp, arr):
    if isinstance(fp, Chem.DataStructs.cDataStructs.ExplicitBitVect):
        DataStructs.ConvertToNumpyArray(fp, arr)
    else:
        output_fp = np.zeros(fp.GetNumBits())
        output_fp = DataStructs.ConvertToNumpyArray(fp, output_fp)
        arr[:] = output_fp[:]
    return arr

def create_mol(df_l, n_bits, method='rdk5'):
    df_l['mol'] = df_l.smiles.apply(Chem.MolFromSmiles)
    df_l['bv'] = df_l.mol.apply(lambda x: calculate_fp(x, method, n_bits))
    df_l['np_bv'] = np.zeros((len(df_l), n_bits)).tolist()
    
    for i, row in df_l.iterrows():
        if row['bv'] is not None:
            df_l.at[i, 'np_bv'] = ConvertToNumpyArray(row['bv'], np.zeros(n_bits))
        else:
            df_l.at[i, 'np_bv'] = np.zeros(n_bits)

### Paso 3: Cargar datos desde un archivo CSV:
Lectura del archivo CSV, con los datos descargados de la pagina ChEMBL

In [ ]:
ChEMBL_df = pd.read_csv('./BACE1.csv')

### Paso 4: Filtrar moléculas
Se filtran las moléculas según el valor de la columna 'Activity_Type' para eliminar las que tienen el valor 'Intermediate'.


In [ ]:
filteredyes_df = ChEMBL_df[(ChEMBL_df['Activity_Type'] != 'Intermediate')]

### Paso 5: Seleccionar columnas relevantes
Se seleccionan las columnas 'Molecule_ChEMBL_ID', 'Activity_Type', y 'Smiles' del DataFrame filtrado. Se crea una nueva columna 'active' que asigna 1 a las moléculas marcadas como 'Active'.

In [ ]:
datosfinales = filteredyes_df[['Molecule_ChEMBL_ID', 'Activity_Type', 'Smiles']].copy()
datosfinales['active'] = 0
datosfinales.loc[datosfinales['Activity_Type'] == "Active", 'active'] = 1

### Paso 6: Contar el número de moléculas activas e inactivas
Se cuentan el número de moléculas activas e inactivas en el DataFrame datosfinales.


In [ ]:
num_actives = len(datosfinales[datosfinales['active'] == 1])
num_inactives = len(datosfinales[datosfinales['active'] == 0])
print('Actives: %d, Inactives: %d' % (num_actives, num_inactives))

### Paso 7: Renombrar columna 'Smiles' a 'smiles'
La columna 'Smiles' se renombra como 'smiles' para asegurar coherencia en el procesamiento.

In [ ]:
datosfinales = datosfinales.rename(columns={'Smiles': 'smiles'})

### Paso 8: Crear moléculas y calcular fingerprints
Se crean moléculas a partir de las cadenas SMILES y se calculan los fingerprints para cada molécula. Los resultados se almacenan en el DataFrame df_new.


In [ ]:
df_new = datosfinales.copy()
df_new = df_new.dropna(subset=['smiles']) 
create_mol(df_new, 500)

### Paso 9: Contar el número de moléculas activas e inactivas después de crear las moléculas
Se vuelve a contar el número de moléculas activas e inactivas en el DataFrame df_new.


In [ ]:
num_actives = len(df_new[df_new['active'] == 1])
num_inactives = len(df_new[df_new['active'] == 0])
print('Actives: %d, Inactives: %d' % (num_actives, num_inactives))


### Paso 10: Seleccionar columnas relevantes para el entrenamiento
Se seleccionan las columnas 'np_bv' y 'active' del DataFrame df_new. Estos datos se utilizarán para el entrenamiento de modelos de aprendizaje automático.


In [ ]:
datosentrenamientoSIN_df = df_new[['np_bv', 'active']].copy()
datosentrenamientoSIN_df.reset_index(drop=True, inplace=True)

### Paso 11: Guardar el DataFrame en un archivo pickle
Finalmente, se guarda el DataFrame datosentrenamientoSIN_df en un archivo pickle llamado 'datosentrenamientoSIN_df.pickle'.


In [ ]:
with open('datosentrenamientoSIN_df.pickle', 'wb') as f:
    pickle.dump(datosentrenamientoSIN_df, f)

Script elaborado por Nelson Alejandro Amaya Orozco, como trabajo de grado, para el grupo de investigacion RamirezLAB